# Limpieza de datos

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('DelayedFlights.csv', low_memory=False)

## Redefinición de nombres de columnas
Como la base de datos tiene nombres de columnas con carácteres especiales, se realiza una estandarización de los nombres de columnas, reemplazando espacios, tildes y otros carácteres especiales que se puedan encontrar.
Esto se hace para evitar problemas futuros en el llamado de las columnas y para facilitar el trabajo del dataset.

In [8]:
# column "Unnamed: 0" = "id"
df['id'] = df['Unnamed: 0']
df.drop(['Unnamed: 0'], axis=1, inplace=True)
print(df.columns)

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'id'],
      dtype='object')


## Valores duplicados
Como se pudo observar en la fase de análisis exploratorio, aparentemente en el dataset no existen valores duplicados. Sin embargo, es importante tener en cuenta que no pueden existir 2 vuelos con el mismo número de vuelo, misma fecha de despegue, mismos origen y destino y mismo carrier. Por esta razón, se ve si existen filas que compartan toda esta información en común

In [2]:
flight_def_cols = ["FlightNum", "Year", "Month", "DayofMonth", "CRSDepTime", "Origin", "Dest", "UniqueCarrier"]
numero_inicial_filas = len(df)
print(f"Número de filas del dataset: {numero_inicial_filas}")
print(f"Número de vuelos diferentes: {len(df.drop_duplicates(subset=flight_def_cols, inplace = False))}")

Número de filas del dataset: 1936758
Número de vuelos diferentes: 1936756


Como se puede ver difieren en 2 filas, por lo cual hay 2 vuelos repetidos. Se filtran estos vuelos:

In [4]:
df.drop_duplicates(subset=flight_def_cols, inplace = True)

## Valores nulos
Como primera medida, se considera como regla dejar fuera las columnas que tienen más de un 50% de datos nulos debido a la falta de información suficiente. 

Como las columnas del dataset que tienen valores nulos son menores del 50%, se mantendrán y se procederá a imputarlos



In [5]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
missing_values = missing_values / df.shape[0] * 100
cols_with_more_than50_mv = missing_values[missing_values > 50]
print(missing_values)

LateAircraftDelay    35.588892
WeatherDelay         35.588892
NASDelay             35.588892
SecurityDelay        35.588892
CarrierDelay         35.588892
ActualElapsedTime     0.433044
ArrDelay              0.433044
AirTime               0.433044
ArrTime               0.367109
TaxiIn                0.367109
TaxiOut               0.023493
CRSElapsedTime        0.010223
TailNum               0.000258
dtype: float64


In [40]:
# IMPUTACIÓN DE VALORES FALTANTES

In [41]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
missing_values = missing_values / df.shape[0] * 100
print(missing_values)

Series([], dtype: float64)


# Creación de nuevas variables



__Cabe destacar que más variables serán creadas en el paso de Visualización de datos conforme sea necesario respecto a las agrupaciones y transformaciones.__

In [12]:
df['DepartureHour'] = (df['DepTime'] // 100).astype(int)
print(df[['DepartureHour']].head())

   DepartureHour
0             20
1              7
2              6
3             18
4             19


In [13]:
df['ArrTime'] = df['ArrTime'].fillna(0)

df['ArrivalHour'] = (df['ArrTime'] // 100).astype(int)

print(df[['ArrivalHour']].head())

   ArrivalHour
0           22
1           10
2            8
3           19
4           21


In [14]:
df['FlightDuration'] = df['ActualElapsedTime'] - df['TaxiIn'] - df['TaxiOut']
print(df[['FlightDuration']].head())


   FlightDuration
0           116.0
1           113.0
2            76.0
3            77.0
4            87.0


In [18]:
conditions = [
    (df['Distance'] <= 300),
    (df['Distance'] <= 800),
    (df['Distance'] > 800)
]
choices = ['Short', 'Medium', 'Long']

# Crea la nueva columna 'DistanceCategory' usando np.select
df['DistanceCategory'] = np.select(conditions, choices, default='Unknown')

# Imprime las primeras filas para verificar los cambios
print(df[['Distance', 'DistanceCategory']].head(20))

    Distance DistanceCategory
0        810             Long
1        810             Long
2        515           Medium
3        515           Medium
4        688           Medium
5       1591             Long
6        828             Long
7        828             Long
8        162            Short
9       1489             Long
10      1489             Long
11       838             Long
12       220            Short
13       220            Short
14       220            Short
15       220            Short
16       220            Short
17      1093             Long
18      1093             Long
19       972             Long


In [19]:
df['DepartureDelay'] = df['DepTime'] - df['CRSDepTime']
print(df[['DepTime', 'CRSDepTime', 'DepartureDelay']].head())


   DepTime  CRSDepTime  DepartureDelay
0   2003.0        1955            48.0
1    754.0         735            19.0
2    628.0         620             8.0
3   1829.0        1755            74.0
4   1940.0        1915            25.0


In [20]:
df['ArrivalDelay'] = df['ArrTime'] - df['CRSArrTime']
print(df[['ArrTime', 'CRSArrTime', 'ArrivalDelay']].head())

   ArrTime  CRSArrTime  ArrivalDelay
0   2211.0        2225         -14.0
1   1002.0        1000           2.0
2    804.0         750          54.0
3   1959.0        1925          34.0
4   2121.0        2110          11.0


In [21]:
df['TotalDelay'] = df['DepartureDelay'] + df['ArrivalDelay']
print(df[['DepartureDelay', 'ArrivalDelay', 'TotalDelay']].head())

   DepartureDelay  ArrivalDelay  TotalDelay
0            48.0         -14.0        34.0
1            19.0           2.0        21.0
2             8.0          54.0        62.0
3            74.0          34.0       108.0
4            25.0          11.0        36.0


In [22]:
df['ScheduledArrivalHour'] = (df['CRSArrTime'] // 100).astype(int)
print(df[['CRSArrTime', 'ScheduledArrivalHour']].head())

   CRSArrTime  ScheduledArrivalHour
0        2225                    22
1        1000                    10
2         750                     7
3        1925                    19
4        2110                    21


# Guardar dataset limpio
A partir de aquí se guardará el dataset limpio y se importará en otros notebooks

In [10]:
df

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,id
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,...,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN,0
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,...,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN,1
2,2008,1,3,4,628.0,620,804.0,750,WN,448,...,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN,2
3,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,...,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0,4
4,2008,1,3,4,1940.0,1915,2121.0,2110,WN,378,...,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936753,2008,12,13,6,1250.0,1220,1617.0,1552,DL,1621,...,18.0,0,N,0,3.0,0.0,0.0,0.0,22.0,7009710
1936754,2008,12,13,6,657.0,600,904.0,749,DL,1631,...,34.0,0,N,0,0.0,57.0,18.0,0.0,0.0,7009717
1936755,2008,12,13,6,1007.0,847,1149.0,1010,DL,1631,...,32.0,0,N,0,1.0,0.0,19.0,0.0,79.0,7009718
1936756,2008,12,13,6,1251.0,1240,1446.0,1437,DL,1639,...,13.0,0,N,0,NaN,NaN,NaN,NaN,NaN,7009726


In [11]:
df.to_csv('DelayedFlightsClean.csv')